<a href="https://colab.research.google.com/github/SonnyDev/llm-apps-langchain/blob/main/LangChain_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Les chaines
L'idée centrale de LangChain est que nous pouvons "chaîner" ensemble différents composants (I/O du modèles, Retrieval, Tools) pour créer des cas d'utilisation plus avancés autour des LLMs.

### 1. Chargerment de la clé d'API

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

### 2. Création d'une chaine simple de type LLMChain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Tu es un historien renommé qui donne des réponses correctes et éloquantes aux questions d'histoire.",
        ),
        ("human", "{question}"),
    ]
)
runnable = prompt | model | StrOutputParser()

In [ ]:
for chunk in runnable.stream({"question": "Qu'est-ce que la révolution francaise"}):
    print(chunk, end="", flush=True)

La Révolution française était un événement majeur qui s'est déroulé de 1789 à 1799 en France. Elle a été caractérisée par une série de bouleversements politiques, sociaux et économiques qui ont radicalement transformé le paysage politique et social de la France.

La Révolution française a été déclenchée par un certain nombre de facteurs, dont les inégalités sociales et économiques, l'oppression du régime monarchique et les idées des Lumières qui prônaient la liberté, l'égalité et la fraternité. Elle a commencé par la prise de la Bastille le 14 juillet 1789, qui a symbolisé la fin de l'absolutisme monarchique et le début d'une nouvelle ère politique.

Pendant la Révolution, la France a connu de nombreux changements radicaux. La monarchie a été abolie et remplacée par une république. Les droits de l'homme ont été proclamés et la noblesse a perdu ses privilèges. De nouvelles lois ont été promulguées pour promouvoir l'égalité sociale et politique. La guillotine est devenue le symbole de la

### 3. Création des chaines séquentielles

In [ ]:
from langchain.prompts import PromptTemplate

synopsis_prompt = PromptTemplate.from_template(
    """Tu es un dramaturge experimenté. Étant donné le titre d'une pièce, \
    ta tache est d'écrire un synopsis pour ce titre. Réponds dans la mème langue que le titre."

Title: {title}
Dramaturge : Voici un synopsis pour la pièce mentionnée ci-dessus:"""
)

review_prompt = PromptTemplate.from_template(
    """Tu es un critique de théâtre pour le journal Le Parisien. \
    Étant donné le synopsis d'une pièce, ton role est d'écrire une critique pour cette pièce.\

Synopsis de la pièce:
{synopsis}
Critique d'un dramaturge du Parisien de la pièce mentionnée ci-dessus :"""
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

llm = ChatOpenAI()
chain = (
    {"synopsis": synopsis_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)
print(chain.invoke({"title": "Médecin malgré lui"},)) #config={'callbacks': [ConsoleCallbackHandler()]}

Dans "Médecin malgré lui", la comédie classique de Molière, le spectateur est transporté dans un monde de quiproquos et d'absurdités où le personnage principal, Sganarelle, se retrouve malgré lui pris pour un médecin renommé. 

Dès le début de la pièce, nous sommes témoins des talents comiques de Sganarelle, interprété brillamment par l'acteur principal. Son jeu d'acteur est à la fois hilarant et attachant, nous faisant rire aux éclats tout en ressentant de l'empathie pour ce paysan alcoolique et paresseux qui se retrouve au cœur de cette farce médicale. 

Les situations dans lesquelles Sganarelle se trouve sont de plus en plus absurdes et comiques au fur et à mesure que l'histoire se développe. Sa tentative d'utiliser des techniques médicales farfelues et des remèdes ridicules pour guérir la fille de Géronte, qui a perdu la parole sans raison apparente, est à la fois désopilante et touchante. 

Les autres personnages de la pièce jouent également un rôle important dans la création de c

In [ ]:
from langchain_core.runnables import RunnablePassthrough # Si je souhaite afficher également le synopsis

synopsis_chain = synopsis_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
chain = {"synopsis": synopsis_chain} | RunnablePassthrough.assign(review=review_chain)
chain.invoke({"title": "Médecin malgré lui"})

{'synopsis': '"Médecin malgré lui" est une comédie classique qui se déroule dans une petite ville de province. L\'histoire tourne autour de Sganarelle, un homme simple et un peu rustre, qui est confondu à tort avec un médecin renommé.\n\nSganarelle, qui est en réalité un bûcheron, se retrouve soudainement plongé dans un monde de médecine et de patients désespérés. Malgré son manque de connaissances médicales, il se voit contraint d\'endosser le rôle de médecin pour sauver la vie d\'une jeune femme, Lucinde.\n\nLa situation se complique lorsque Géronte, le père de Lucinde, exige que Sganarelle guérisse sa fille et menace de le punir s\'il échoue. Bien que réticent au début, Sganarelle commence à jouer le jeu et à donner des conseils médicaux farfelus, convaincu qu\'il ne peut pas faire pire que les vrais médecins.\n\nAu fur et à mesure que la pièce avance, Sganarelle se retrouve dans des situations de plus en plus absurdes et hilarantes, jonglant entre les remèdes improbables et les dia

### 4. Chaines de Transformation
En utilisant LCEL, la créations de chaines de transformation est triviale étant donné qu'on peut ajouter des fonctions dans n'importe quel objet RunnableSequence

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Résume ce texte:

{output_text}

Résumé:"""
)

In [ ]:
with open("license.txt") as f:
    license_file = f.read()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

runnable = (
    {"output_text": lambda text: "\n\n".join(text.split("\n\n")[:4])}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)
runnable.invoke(license_file) #config={'callbacks': [ConsoleCallbackHandler()]}

"Ce texte concerne le logiciel WordPress, qui est un système de publication de contenu sur le web. Il précise que le programme est sous licence GNU General Public License, ce qui signifie qu'il peut être redistribué et modifié librement. Il est distribué sans garantie, mais avec l'espoir qu'il sera utile."

### 5. Router Chains
Le routage permet de créer des chaînes non déterministes où la sortie d'une étape précédente définit l'étape suivante. Le routage aide à fournir une structure et une cohérence autour des interactions avec les LLMs.

In [ ]:
from langchain.prompts import PromptTemplate

physics_template = """Tu es un professeur de physique très intelligent.\
Tu es excellent pour répondre aux questions sur la physique de manière concise et facile à comprendre.\
Lorsque tu ne connais pas la réponse à une question, tu admets que tu ne sais pas.

Voici une question :
{input}"""
physics_prompt = PromptTemplate.from_template(physics_template)

math_template = """Tu es un très bon mathématicien. \
Tu es excellent pour répondre aux questions de mathématiques. \
Tu es si bon parce que tu es capable de décomposer les problèmes difficiles en leurs composantes, \
de répondre à ces composantes, puis de les assembler pour répondre à la question plus large.

Voici une question :
{input}"""
math_prompt = PromptTemplate.from_template(math_template)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch
from langchain.callbacks.tracers import ConsoleCallbackHandler

In [ ]:
general_prompt = PromptTemplate.from_template(
    "Tu es un assistant très utile. Réponds à la question aussi précisément que possible.\n\n{input}"
)
prompt_branch = RunnableBranch(
    (lambda x: x["topic"] == "math", math_prompt),
    (lambda x: x["topic"] == "physics", physics_prompt),
    general_prompt,
)

In [ ]:
from typing import Literal

from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.pydantic_v1 import BaseModel


class TopicClassifier(BaseModel):
    "Classification du sujet de la question de l'utilisateur"

    topic: Literal["math", "physics", "general"]
    "Le sujet de l'utilisateur peut etre 'math', 'physics' ou 'general'."


classifier_function = convert_pydantic_to_openai_function(TopicClassifier)
llm = ChatOpenAI().bind(
    functions=[classifier_function], function_call={"name": "TopicClassifier"}
)
parser = PydanticAttrOutputFunctionsParser(
    pydantic_schema=TopicClassifier, attr_name="topic"
)
classifier_chain = llm | parser

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
    | prompt_branch
    | ChatOpenAI()
    | StrOutputParser()
)

In [ ]:
print(final_chain.invoke(
    {
        "input": "Qu'est-ce que la loi de la gravité ?"
    }, config={'callbacks': [ConsoleCallbackHandler()]}
))

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Qu'est-ce que la loi de la gravité ?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign] Entering Chain run with input:
{
  "input": "Qu'est-ce que la loi de la gravité ?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign > 3:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "Qu'est-ce que la loi de la gravité ?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign > 3:chain:RunnableParallel > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Qu'est-ce que la loi de la gravité ?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign > 3:chain:RunnableParallel > 4:chain:RunnableSequence > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "Qu'est-ce que la loi de la gravité ?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableAssign > 3:chain:RunnableParallel > 4:chain:RunnableSe